Wichtig:
- World Eintrag löschen (Ja)
- einheitliche column names für alle Tabellen (Ja)
- Länder bei Apple,Google,Internet löschen (Ja)
- Extra Tabelle nur für die Länder


- Schema für die Tabellen in Postgresql einrichten
- Exportieren

- Interpolation bei Google Dataset anpassen (Ja)
- NaNs der Flüge mit 0 füllen (Ja)
- Lösung für iso-code von Namibia finden (Ja)

Vlt:
- Multipolygons der Länder in Größe reduzieren (Nein)
- Dates einschränken(z.B. bis Ende 2020) (Nein)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

# Apple Dataset

In [3]:
df_apple = pd.read_csv(r'GeospacialDBs_Data/applemobilitytrends-2021-02-07.csv', 
                       header=0,  dtype={"alternative_name": str})
df_apple.head(5)

,geo_type,region,transportation_type,alternative_name,sub-region,country,2020-01-13,2020-01-14,2020-01-15,2020-01-16,...,2021-01-29,2021-01-30,2021-01-31,2021-02-01,2021-02-02,2021-02-03,2021-02-04,2021-02-05,2021-02-06,2021-02-07
0,country/region,Albania,driving,NaN,NaN,NaN,100.0,95.30,101.43,97.20,...,116.21,132.17,132.96,110.48,112.46,112.31,110.95,121.39,150.97,144.41
1,country/region,Albania,walking,NaN,NaN,NaN,100.0,100.68,98.93,98.46,...,134.84,151.51,115.99,132.11,140.51,137.91,145.88,145.37,174.50,135.57
2,country/region,Argentina,driving,NaN,NaN,NaN,100.0,97.07,102.45,111.21,...,101.35,96.44,73.38,79.07,78.49,82.45,88.12,104.89,115.25,83.65
3,country/region,Argentina,walking,NaN,NaN,NaN,100.0,95.11,101.37,112.67,...,78.62,62.64,46.36,61.61,68.87,74.70,77.16,85.55,81.70,53.45
4,country/region,Australia,driving,AU,NaN,NaN,100.0,102.98,104.21,108.63,...,111.23,91.77,97.74,100.00,102.65,105.44,109.94,113.49,93.67,101.23


In [4]:
df_apple["geo_type"].unique()

array(['country/region', 'city', 'sub-region', 'county'], dtype=object)

In [5]:
df_apple_countries = df_apple[df_apple["geo_type"] == 'country/region'].copy()
df_apple_countries.drop(["geo_type","sub-region","country","alternative_name"], inplace=True, axis=1)
df_apple_countries

,region,transportation_type,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20,...,2021-01-29,2021-01-30,2021-01-31,2021-02-01,2021-02-02,2021-02-03,2021-02-04,2021-02-05,2021-02-06,2021-02-07
0,Albania,driving,100.0,95.30,101.43,97.20,103.55,112.67,104.83,94.39,...,116.21,132.17,132.96,110.48,112.46,112.31,110.95,121.39,150.97,144.41
1,Albania,walking,100.0,100.68,98.93,98.46,100.85,100.13,82.13,95.65,...,134.84,151.51,115.99,132.11,140.51,137.91,145.88,145.37,174.50,135.57
2,Argentina,driving,100.0,97.07,102.45,111.21,118.45,124.01,95.44,95.13,...,101.35,96.44,73.38,79.07,78.49,82.45,88.12,104.89,115.25,83.65
3,Argentina,walking,100.0,95.11,101.37,112.67,116.72,114.14,84.54,101.37,...,78.62,62.64,46.36,61.61,68.87,74.70,77.16,85.55,81.70,53.45
4,Australia,driving,100.0,102.98,104.21,108.63,109.08,89.00,99.35,103.53,...,111.23,91.77,97.74,100.00,102.65,105.44,109.94,113.49,93.67,101.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,United States,walking,100.0,99.59,101.87,100.60,121.36,132.38,100.10,94.79,...,113.83,122.52,82.91,82.97,87.95,95.05,98.17,119.41,126.94,82.96
149,Uruguay,driving,100.0,96.84,92.90,103.48,114.91,115.03,99.48,93.30,...,81.04,68.54,54.73,65.78,71.03,65.97,75.59,95.48,92.69,73.64
150,Uruguay,walking,100.0,94.90,78.04,100.46,105.04,98.78,76.52,89.94,...,72.32,46.29,39.97,57.40,68.66,64.97,75.19,86.55,74.38,50.98
151,Vietnam,driving,100.0,104.96,108.67,107.09,121.07,130.40,123.99,122.77,...,97.61,102.81,87.52,84.21,82.70,79.40,82.43,93.98,102.54,95.09


In [6]:
df_list = []
for region in df_apple_countries["region"].unique():
    for transportation_type in df_apple_countries[df_apple_countries["region"] == region]["transportation_type"].unique():
        series_sub = df_apple_countries[(df_apple_countries["region"] == region) & (df_apple_countries["transportation_type"] == transportation_type)].iloc[:, 2:].transpose()
        df_list += [{'date': timestamp, 'country_region': region, 'transportation_type': transportation_type, 'percent_change_from_baseline': value[0]} for timestamp, value in zip(series_sub.index, series_sub.values)]
df_apple_reshaped = pd.DataFrame(df_list)

df_apple_reshaped

,date,country_region,transportation_type,percent_change_from_baseline
0,2020-01-13,Albania,driving,100.00
1,2020-01-14,Albania,driving,95.30
2,2020-01-15,Albania,driving,101.43
3,2020-01-16,Albania,driving,97.20
4,2020-01-17,Albania,driving,103.55
...,...,...,...,...
59971,2021-02-03,Vietnam,walking,107.06
59972,2021-02-04,Vietnam,walking,112.53
59973,2021-02-05,Vietnam,walking,124.88
59974,2021-02-06,Vietnam,walking,129.99


In [7]:
for i in range(len(df_apple_reshaped)):
    df_apple_reshaped.at[i,'percent_change_from_baseline']-=100

df_apple_reshaped

,date,country_region,transportation_type,percent_change_from_baseline
0,2020-01-13,Albania,driving,0.00
1,2020-01-14,Albania,driving,-4.70
2,2020-01-15,Albania,driving,1.43
3,2020-01-16,Albania,driving,-2.80
4,2020-01-17,Albania,driving,3.55
...,...,...,...,...
59971,2021-02-03,Vietnam,walking,7.06
59972,2021-02-04,Vietnam,walking,12.53
59973,2021-02-05,Vietnam,walking,24.88
59974,2021-02-06,Vietnam,walking,29.99


### Fill in Nans with with interpolate


In [8]:
Nans=df_apple_reshaped[df_apple_reshaped.isnull().any(axis=1)]
Nans["date"].unique()

array(['2020-05-11', '2020-05-12'], dtype=object)

In [9]:
percent_change=df_apple_reshaped['percent_change_from_baseline'].interpolate(method='quadratic')
df_apple_reshaped['percent_change_from_baseline']=percent_change
df_apple_reshaped

,date,country_region,transportation_type,percent_change_from_baseline
0,2020-01-13,Albania,driving,0.00
1,2020-01-14,Albania,driving,-4.70
2,2020-01-15,Albania,driving,1.43
3,2020-01-16,Albania,driving,-2.80
4,2020-01-17,Albania,driving,3.55
...,...,...,...,...
59971,2021-02-03,Vietnam,walking,7.06
59972,2021-02-04,Vietnam,walking,12.53
59973,2021-02-05,Vietnam,walking,24.88
59974,2021-02-06,Vietnam,walking,29.99


## Add iso-country codes

In [10]:
df_iso_codes = pd.read_csv(r'GeospacialDBs_Data/wikipedia-iso-country-codes.csv', header=0, sep=",")
df_iso_codes

,English short name lower case,Alpha-2 code,Alpha-3 code,Numeric code,ISO 3166-2
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX
2,Albania,AL,ALB,8,ISO 3166-2:AL
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ
4,American Samoa,AS,ASM,16,ISO 3166-2:AS
...,...,...,...,...,...
241,Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF
242,Western Sahara,EH,ESH,732,ISO 3166-2:EH
243,Yemen,YE,YEM,887,ISO 3166-2:YE
244,Zambia,ZM,ZMB,894,ISO 3166-2:ZM


In [11]:
df_iso_codes.set_index('English short name lower case',inplace=True)
df_iso_codes

,Alpha-2 code,Alpha-3 code,Numeric code,ISO 3166-2
English short name lower case,,,,
Afghanistan,AF,AFG,4,ISO 3166-2:AF
Åland Islands,AX,ALA,248,ISO 3166-2:AX
Albania,AL,ALB,8,ISO 3166-2:AL
Algeria,DZ,DZA,12,ISO 3166-2:DZ
American Samoa,AS,ASM,16,ISO 3166-2:AS
...,...,...,...,...
Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF
Western Sahara,EH,ESH,732,ISO 3166-2:EH
Yemen,YE,YEM,887,ISO 3166-2:YE


In [12]:
countries={}
for i in range(len(df_apple_reshaped)):
    tmp=df_apple_reshaped['country_region'][i]
    if tmp not in df_iso_codes.index:
        if tmp not in countries:
            countries[tmp]=1

countries  

{'Republic of Korea': 1, 'Russia': 1, 'Taiwan': 1, 'Vietnam': 1}

In [13]:
for i in df_iso_codes.index:
    print(i)

Afghanistan
Åland Islands
Albania
Algeria
American Samoa
Andorra
Angola
Anguilla
Antarctica
Antigua and Barbuda
Argentina
Armenia
Aruba
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bermuda
Bhutan
Bolivia, Plurinational State of
Bosnia and Herzegovina
Botswana
Bouvet Island
Brazil
British Indian Ocean Territory
Brunei Darussalam
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Cape Verde
Cayman Islands
Central African Republic
Chad
Chile
China
Christmas Island
Cocos (Keeling) Islands
Colombia
Comoros
Congo
Congo, the Democratic Republic of the
Cook Islands
Costa Rica
Côte d'Ivoire
Croatia
Cuba
Cyprus
Czech Republic
Denmark
Djibouti
Dominica
Dominican Republic
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Falkland Islands (Malvinas)
Faroe Islands
Fiji
Finland
France
French Guiana
French Polynesia
French Southern Territories
Gabon
Gambia
Georgia
Germany
Ghana
Gibraltar
Greece
Greenland
Grenada
Guadeloupe
Guam
Gu

In [14]:
#translate the missing ones
countries['Republic of Korea']="Korea, Republic of"
countries['Russia']='Russian Federation'
countries['Taiwan']='Taiwan, Province of China'
countries['Vietnam']='Viet Nam'

countries


{'Republic of Korea': 'Korea, Republic of',
 'Russia': 'Russian Federation',
 'Taiwan': 'Taiwan, Province of China',
 'Vietnam': 'Viet Nam'}

In [15]:
df_apple_reshaped['country_region_code']='Placeholder'
df_apple_reshaped

,date,country_region,transportation_type,percent_change_from_baseline,country_region_code
0,2020-01-13,Albania,driving,0.00,Placeholder
1,2020-01-14,Albania,driving,-4.70,Placeholder
2,2020-01-15,Albania,driving,1.43,Placeholder
3,2020-01-16,Albania,driving,-2.80,Placeholder
4,2020-01-17,Albania,driving,3.55,Placeholder
...,...,...,...,...,...
59971,2021-02-03,Vietnam,walking,7.06,Placeholder
59972,2021-02-04,Vietnam,walking,12.53,Placeholder
59973,2021-02-05,Vietnam,walking,24.88,Placeholder
59974,2021-02-06,Vietnam,walking,29.99,Placeholder


In [16]:
for i in range(len(df_apple_reshaped)):
    tmp=df_apple_reshaped['country_region'][i]
    if tmp not in set(df_iso_codes.index):
        tmp=countries[tmp]
    df_apple_reshaped['country_region_code'][i]=df_iso_codes['Alpha-2 code'][tmp]

df_apple_reshaped
    
        

C:\Users\Rolle\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,date,country_region,transportation_type,percent_change_from_baseline,country_region_code
0,2020-01-13,Albania,driving,0.00,AL
1,2020-01-14,Albania,driving,-4.70,AL
2,2020-01-15,Albania,driving,1.43,AL
3,2020-01-16,Albania,driving,-2.80,AL
4,2020-01-17,Albania,driving,3.55,AL
...,...,...,...,...,...
59971,2021-02-03,Vietnam,walking,7.06,VN
59972,2021-02-04,Vietnam,walking,12.53,VN
59973,2021-02-05,Vietnam,walking,24.88,VN
59974,2021-02-06,Vietnam,walking,29.99,VN


In [18]:
df_apple_reshaped['country_region_code'].isnull().sum()

0

In [19]:
df_apple_reshaped.drop(columns=['country_region'],inplace=True)
df_apple_reshaped

,date,transportation_type,percent_change_from_baseline,country_region_code
0,2020-01-13,driving,0.00,AL
1,2020-01-14,driving,-4.70,AL
2,2020-01-15,driving,1.43,AL
3,2020-01-16,driving,-2.80,AL
4,2020-01-17,driving,3.55,AL
...,...,...,...,...
59971,2021-02-03,walking,7.06,VN
59972,2021-02-04,walking,12.53,VN
59973,2021-02-05,walking,24.88,VN
59974,2021-02-06,walking,29.99,VN


In [20]:
df_apple_reshaped.to_csv(r"GeospacialDBs_Data/AppleMobilityData_reshaped.csv", index = False)

# Google Dataset

In [21]:
df_google = pd.read_csv('GeospacialDBs_Data/Global_Mobility_Report.csv', header=0,  dtype={"metro_area": str})
df_google

C:\Users\Rolle\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4126753,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2021-01-27,NaN,NaN,NaN,NaN,-31.0,NaN
4126754,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2021-01-28,NaN,NaN,NaN,NaN,-32.0,NaN
4126755,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2021-01-29,NaN,NaN,NaN,NaN,-36.0,NaN
4126756,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2021-02-01,NaN,NaN,NaN,NaN,-34.0,NaN


In [22]:
df_google.drop(["sub_region_1","sub_region_2","metro_area","iso_3166_2_code","census_fips_code"], inplace=True, axis=1)
df_google.head(5)

,country_region_code,country_region,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


In [23]:
df_google[df_google.isnull().any(axis=1)]

,country_region_code,country_region,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
777,AE,United Arab Emirates,2020-04-24,-62.0,-34.0,-92.0,NaN,-40.0,22.0
784,AE,United Arab Emirates,2020-05-01,-54.0,-31.0,-85.0,NaN,-35.0,20.0
819,AE,United Arab Emirates,2020-06-05,-35.0,-17.0,-58.0,NaN,-18.0,16.0
833,AE,United Arab Emirates,2020-06-19,-36.0,-19.0,-68.0,NaN,-16.0,15.0
840,AE,United Arab Emirates,2020-06-26,-27.0,-14.0,-56.0,NaN,-12.0,13.0
...,...,...,...,...,...,...,...,...,...
4126753,ZW,Zimbabwe,2021-01-27,NaN,NaN,NaN,NaN,-31.0,NaN
4126754,ZW,Zimbabwe,2021-01-28,NaN,NaN,NaN,NaN,-32.0,NaN
4126755,ZW,Zimbabwe,2021-01-29,NaN,NaN,NaN,NaN,-36.0,NaN
4126756,ZW,Zimbabwe,2021-02-01,NaN,NaN,NaN,NaN,-34.0,NaN


In [24]:
df_google.isnull().sum()

country_region_code                                      2627
country_region                                              0
date                                                        0
retail_and_recreation_percent_change_from_baseline    1530523
grocery_and_pharmacy_percent_change_from_baseline     1620955
parks_percent_change_from_baseline                    2147682
transit_stations_percent_change_from_baseline         2040468
workplaces_percent_change_from_baseline                193599
residential_percent_change_from_baseline              1731784
dtype: int64

In [25]:
df_google[df_google['country_region_code'].isnull()]
#Namibias iso country code is read as NaN

,country_region_code,country_region,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
2167539,NaN,Namibia,2020-02-15,-5.0,-8.0,2.0,-8.0,2.0,3.0
2167540,NaN,Namibia,2020-02-16,-12.0,-10.0,4.0,-9.0,-1.0,2.0
2167541,NaN,Namibia,2020-02-17,-13.0,-14.0,-3.0,-17.0,6.0,1.0
2167542,NaN,Namibia,2020-02-18,-15.0,-14.0,-6.0,-11.0,6.0,2.0
2167543,NaN,Namibia,2020-02-19,-13.0,-14.0,-5.0,-15.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...
2170161,NaN,Namibia,2021-02-01,NaN,NaN,NaN,NaN,-13.0,NaN
2170162,NaN,Namibia,2021-02-02,NaN,NaN,NaN,NaN,-14.0,NaN
2170163,NaN,Namibia,2020-04-13,NaN,NaN,NaN,NaN,-59.0,NaN
2170164,NaN,Namibia,2020-12-25,NaN,NaN,NaN,NaN,-75.0,NaN


In [26]:
#Namibias iso code is read as NaN, replace it
for i in range(len(df_google)):
    tmp=df_google['country_region_code'][i]
    if pd.isnull(tmp):
        df_google['country_region_code'][i]='NA'

C:\Users\Rolle\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [27]:
df_google['country_region_code'].isnull().sum()

0

In [28]:
df_slices=[]
tmp_code=df_google['country_region_code'][0]
tmp_num=0
for i in range(len(df_google)):
    if df_google['country_region_code'][i]!=tmp_code:
        df_slices.append(df_google.iloc[tmp_num:i])
        tmp_num=i
    tmp_code=df_google['country_region_code'][i]
df_slices.append(df_google.iloc[tmp_num:len(df_google)-1])

df_slices[100]

,country_region_code,country_region,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
2480378,PK,Pakistan,2020-02-15,3.0,3.0,8.0,4.0,2.0,0.0
2480379,PK,Pakistan,2020-02-16,3.0,3.0,8.0,4.0,2.0,0.0
2480380,PK,Pakistan,2020-02-17,8.0,8.0,11.0,9.0,3.0,-1.0
2480381,PK,Pakistan,2020-02-18,4.0,4.0,9.0,6.0,4.0,-1.0
2480382,PK,Pakistan,2020-02-19,0.0,0.0,6.0,3.0,4.0,-1.0
...,...,...,...,...,...,...,...,...,...
2487757,PK,Pakistan,2021-01-29,-8.0,21.0,1.0,12.0,-2.0,0.0
2487758,PK,Pakistan,2021-01-30,-6.0,29.0,2.0,13.0,7.0,-2.0
2487759,PK,Pakistan,2021-01-31,0.0,29.0,9.0,23.0,7.0,-3.0
2487760,PK,Pakistan,2021-02-01,-1.0,32.0,6.0,14.0,-1.0,-3.0


In [29]:
# fill in NaNs in percent change with interpolate
for slices in df_slices:
    for col in slices.columns: 
        if col!="country_region_code" and col!="country_region" and col!="date":
            percent_change=slices[col].interpolate(method='quadratic')
            slices[col]=percent_change

C:\Users\Rolle\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
df_slices[0].isnull().sum()

country_region_code                                     0
country_region                                          0
date                                                    0
retail_and_recreation_percent_change_from_baseline      0
grocery_and_pharmacy_percent_change_from_baseline       0
parks_percent_change_from_baseline                    347
transit_stations_percent_change_from_baseline         347
workplaces_percent_change_from_baseline                 0
residential_percent_change_from_baseline                0
dtype: int64

In [31]:
df_google=pd.concat(df_slices)
df_google

,country_region_code,country_region,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...
4126752,ZW,Zimbabwe,2021-01-26,NaN,NaN,NaN,NaN,-30.0,NaN
4126753,ZW,Zimbabwe,2021-01-27,NaN,NaN,NaN,NaN,-31.0,NaN
4126754,ZW,Zimbabwe,2021-01-28,NaN,NaN,NaN,NaN,-32.0,NaN
4126755,ZW,Zimbabwe,2021-01-29,NaN,NaN,NaN,NaN,-36.0,NaN


In [32]:
df_google.isnull().sum()

country_region_code                                       0
country_region                                            0
date                                                      0
retail_and_recreation_percent_change_from_baseline    14487
grocery_and_pharmacy_percent_change_from_baseline     21485
parks_percent_change_from_baseline                    27016
transit_stations_percent_change_from_baseline         33229
workplaces_percent_change_from_baseline                1188
residential_percent_change_from_baseline              36176
dtype: int64

In [33]:
#drop the rows with the remaining NaNs
df_google.dropna(inplace=True)
df_google.isnull().sum()

country_region_code                                   0
country_region                                        0
date                                                  0
retail_and_recreation_percent_change_from_baseline    0
grocery_and_pharmacy_percent_change_from_baseline     0
parks_percent_change_from_baseline                    0
transit_stations_percent_change_from_baseline         0
workplaces_percent_change_from_baseline               0
residential_percent_change_from_baseline              0
dtype: int64

In [34]:
df_google.drop(columns=['country_region'],inplace=True)
df_google

,country_region_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0
...,...,...,...,...,...,...,...,...
4122148,ZW,2021-01-29,-36.0,-18.0,-40.0,-58.0,-37.0,21.0
4122149,ZW,2021-01-30,-34.0,-8.0,-37.0,-63.0,-25.0,12.0
4122150,ZW,2021-01-31,-33.0,-11.0,-29.0,-62.0,-17.0,13.0
4122151,ZW,2021-02-01,-30.0,-18.0,-25.0,-55.0,-36.0,19.0


In [35]:
df_google.to_csv(r"GeospacialDBs_Data/Global_Mobility_Report_reshaped.csv", index = False)

# Internet Speed Dataset

In [38]:
df_internet = pd.read_csv('GeospacialDBs_Data/2020-07-20 Ookla Speedtest Tracking COVID-19.csv' , header=0, sep=";")#,  dtype={"metro_area": str})
df_internet

,Place,Result Week Start,Result Week End,Mobile Test Change vs 12 Week Avg,Mobile Download % Change vs Base Week,Mobile Average Download Mbps,Mobile Median Download % Change vs Base Week,Mobile Median Download Mbps,Fixed Test Change vs 12 Week Avg,Fixed Download % Change vs Base Week,Fixed Average Download Mbps,Fixed Median Download % Change vs Base Week,Fixed Median Download Mbps
0,Afghanistan,16.12.2019,22.12.2019,"-0,1014","0,00439","6,86","0,11483","4,66","-0,0664","-0,15459","7,00","-0,01031","1,92"
1,Afghanistan,23.12.2019,29.12.2019,"-0,1998","0,13909","7,78","0,15072","4,81","-0,1493","-0,17391","6,84","0,1134","2,16"
2,Afghanistan,30.12.2019,05.01.2020,"-0,0812","-0,02782","6,64","0,01914","4,26","-0,226","-0,14734","7,06","0,01546","1,97"
3,Afghanistan,06.01.2020,12.01.2020,"-0,2085","0,03221","7,05","0,08134","4,52","-0,2162","-0,04469","7,91","0,07732","2,09"
4,Afghanistan,13.01.2020,19.01.2020,"-0,1034","0,04978","7,17","0,09809","4,59","-0,1126","-0,18237","6,77","0,01031","1,96"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3591,Global,15.06.2020,21.06.2020,"-0,0377","0,07398","35,13","0,02727","18,46","-0,0973","0,01729","77,66","-0,00138","36,16"
3592,Global,22.06.2020,28.06.2020,"-0,0548","0,07123","35,04","0,02115","18,35","-0,1169","0,01755","77,68","-0,00414","36,06"
3593,Global,29.06.2020,05.07.2020,"-0,0253","0,08866","35,61","0,03951","18,68","-0,1027","0,03144","78,74","0,00387","36,35"
3594,Global,06.07.2020,12.07.2020,"-0,0234","0,06053","34,69","0,00278","18,02","-0,1024","0,04716","79,94","-0,00083","36,18"


In [39]:
df_internet.rename(columns={'Place':'country_region'}, inplace=True)

In [40]:
for i in range(len(df_internet)):
    tmp=df_internet['country_region'][i]
    if tmp not in df_iso_codes.index:
        if tmp not in countries:
            countries[tmp]=1

countries

{'Republic of Korea': 'Korea, Republic of',
 'Russia': 'Russian Federation',
 'Taiwan': 'Taiwan, Province of China',
 'Vietnam': 'Viet Nam',
 'Bolivia': 1,
 'Czechia': 1,
 'Hong Kong (SAR)': 1,
 'Laos': 1,
 'Libya': 1,
 'Macau (SAR)': 1,
 'Moldova': 1,
 'Myanmar (Burma)': 1,
 'North Macedonia': 1,
 'Syria': 1,
 'Tanzania': 1,
 'Venezuela': 1,
 'Global': 1}

In [41]:
for i in df_iso_codes.index:
    print(i)

Afghanistan
Åland Islands
Albania
Algeria
American Samoa
Andorra
Angola
Anguilla
Antarctica
Antigua and Barbuda
Argentina
Armenia
Aruba
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bermuda
Bhutan
Bolivia, Plurinational State of
Bosnia and Herzegovina
Botswana
Bouvet Island
Brazil
British Indian Ocean Territory
Brunei Darussalam
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Cape Verde
Cayman Islands
Central African Republic
Chad
Chile
China
Christmas Island
Cocos (Keeling) Islands
Colombia
Comoros
Congo
Congo, the Democratic Republic of the
Cook Islands
Costa Rica
Côte d'Ivoire
Croatia
Cuba
Cyprus
Czech Republic
Denmark
Djibouti
Dominica
Dominican Republic
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Falkland Islands (Malvinas)
Faroe Islands
Fiji
Finland
France
French Guiana
French Polynesia
French Southern Territories
Gabon
Gambia
Georgia
Germany
Ghana
Gibraltar
Greece
Greenland
Grenada
Guadeloupe
Guam
Gu

In [42]:
countries['Bolivia']='Bolivia, Plurinational State of'
countries['Czechia']='Czech Republic'
countries['Hong Kong (SAR)']='Hong Kong'
countries['Laos']="Lao People's Democratic Republic"
countries['Libya']='Libyan Arab Jamahiriya'
countries['Macau (SAR)']='Macao'
countries['Moldova']='Moldova, Republic of'
countries['Myanmar (Burma)']='Myanmar'
countries['North Macedonia']='Macedonia, the former Yugoslav Republic of'
countries['Syria']='Syrian Arab Republic'
countries['Tanzania']='Tanzania, United Republic of'
countries['Venezuela']='Venezuela, Bolivarian Republic of'
del countries['Global'] #not a country

countries

{'Republic of Korea': 'Korea, Republic of',
 'Russia': 'Russian Federation',
 'Taiwan': 'Taiwan, Province of China',
 'Vietnam': 'Viet Nam',
 'Bolivia': 'Bolivia, Plurinational State of',
 'Czechia': 'Czech Republic',
 'Hong Kong (SAR)': 'Hong Kong',
 'Laos': "Lao People's Democratic Republic",
 'Libya': 'Libyan Arab Jamahiriya',
 'Macau (SAR)': 'Macao',
 'Moldova': 'Moldova, Republic of',
 'Myanmar (Burma)': 'Myanmar',
 'North Macedonia': 'Macedonia, the former Yugoslav Republic of',
 'Syria': 'Syrian Arab Republic',
 'Tanzania': 'Tanzania, United Republic of',
 'Venezuela': 'Venezuela, Bolivarian Republic of'}

In [43]:
df_internet['country_region_code']='Placeholder'

In [44]:
global_num=0
for i in range(len(df_internet)):
    tmp=df_internet['country_region'][i]   
    if tmp not in set(df_iso_codes.index):
        if tmp in countries:
            tmp=countries[tmp]
    if tmp!='Global': 
        df_internet['country_region_code'][i]=df_iso_codes['Alpha-2 code'][tmp]
    else:
        global_num=i
        break
        
#delete World rows        
df_internet.drop([x for x in range(global_num,len(df_internet))],inplace=True)
df_internet

,country_region,Result Week Start,Result Week End,Mobile Test Change vs 12 Week Avg,Mobile Download % Change vs Base Week,Mobile Average Download Mbps,Mobile Median Download % Change vs Base Week,Mobile Median Download Mbps,Fixed Test Change vs 12 Week Avg,Fixed Download % Change vs Base Week,Fixed Average Download Mbps,Fixed Median Download % Change vs Base Week,Fixed Median Download Mbps,country_region_code
0,Afghanistan,16.12.2019,22.12.2019,"-0,1014","0,00439","6,86","0,11483","4,66","-0,0664","-0,15459","7,00","-0,01031","1,92",AF
1,Afghanistan,23.12.2019,29.12.2019,"-0,1998","0,13909","7,78","0,15072","4,81","-0,1493","-0,17391","6,84","0,1134","2,16",AF
2,Afghanistan,30.12.2019,05.01.2020,"-0,0812","-0,02782","6,64","0,01914","4,26","-0,226","-0,14734","7,06","0,01546","1,97",AF
3,Afghanistan,06.01.2020,12.01.2020,"-0,2085","0,03221","7,05","0,08134","4,52","-0,2162","-0,04469","7,91","0,07732","2,09",AF
4,Afghanistan,13.01.2020,19.01.2020,"-0,1034","0,04978","7,17","0,09809","4,59","-0,1126","-0,18237","6,77","0,01031","1,96",AF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3560,Vietnam,15.06.2020,21.06.2020,"-0,2156","0,02889","32,76","0,04435","26,61","-0,3743","0,29034","53,82","0,23976","43,28",VN
3561,Vietnam,22.06.2020,28.06.2020,"-0,1991","0,03298","32,89","0,03964","26,49","-0,3613","0,27787","53,30","0,23059","42,96",VN
3562,Vietnam,29.06.2020,05.07.2020,"-0,2105","0,02513","32,64","0,02865","26,21","-0,3208","0,29274","53,92","0,19908","41,86",VN
3563,Vietnam,06.07.2020,12.07.2020,"-0,0951","-0,00031","31,83","0,00746","25,67","-0,2469","0,31359","54,79","0,16729","40,75",VN


In [45]:
df_internet.isnull().any(axis=1).sum()

0

In [46]:
df_internet.drop(columns=['country_region'],inplace=True)
df_internet

,Result Week Start,Result Week End,Mobile Test Change vs 12 Week Avg,Mobile Download % Change vs Base Week,Mobile Average Download Mbps,Mobile Median Download % Change vs Base Week,Mobile Median Download Mbps,Fixed Test Change vs 12 Week Avg,Fixed Download % Change vs Base Week,Fixed Average Download Mbps,Fixed Median Download % Change vs Base Week,Fixed Median Download Mbps,country_region_code
0,16.12.2019,22.12.2019,"-0,1014","0,00439","6,86","0,11483","4,66","-0,0664","-0,15459","7,00","-0,01031","1,92",AF
1,23.12.2019,29.12.2019,"-0,1998","0,13909","7,78","0,15072","4,81","-0,1493","-0,17391","6,84","0,1134","2,16",AF
2,30.12.2019,05.01.2020,"-0,0812","-0,02782","6,64","0,01914","4,26","-0,226","-0,14734","7,06","0,01546","1,97",AF
3,06.01.2020,12.01.2020,"-0,2085","0,03221","7,05","0,08134","4,52","-0,2162","-0,04469","7,91","0,07732","2,09",AF
4,13.01.2020,19.01.2020,"-0,1034","0,04978","7,17","0,09809","4,59","-0,1126","-0,18237","6,77","0,01031","1,96",AF
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3560,15.06.2020,21.06.2020,"-0,2156","0,02889","32,76","0,04435","26,61","-0,3743","0,29034","53,82","0,23976","43,28",VN
3561,22.06.2020,28.06.2020,"-0,1991","0,03298","32,89","0,03964","26,49","-0,3613","0,27787","53,30","0,23059","42,96",VN
3562,29.06.2020,05.07.2020,"-0,2105","0,02513","32,64","0,02865","26,21","-0,3208","0,29274","53,92","0,19908","41,86",VN
3563,06.07.2020,12.07.2020,"-0,0951","-0,00031","31,83","0,00746","25,67","-0,2469","0,31359","54,79","0,16729","40,75",VN


In [47]:
df_internet.to_csv(r"GeospacialDBs_Data/Ookla Speedtest Tracking COVID-19_reshaped.csv", index = False)

# Flight Data

In [48]:
df_airport_codes = pd.read_csv(r'GeospacialDBs_Data/airport-codes.csv' 
, header=0, sep=",")#,  dtype={"number": str})
df_airport_codes.head(5)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"40.07080078125, -74.93360137939453"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"38.704022, -101.473911"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"59.94919968, -151.695999146"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"34.86479949951172, -86.77030181884766"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"35.6087, -91.254898"


In [49]:
df_airport_codes[df_airport_codes['iso_country'].isnull()]

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
21432,FYAA,small_airport,Ai-Ais Airport,2000.0,AF,NaN,NA-KU,Ai-Ais,FYAA,AIW,NaN,"-27.995, 17.5966"
21433,FYAB,small_airport,Aroab B Airport,3235.0,AF,NaN,NA-KA,Aroab,FYAB,NaN,NaN,"-26.776100158691406, 19.633100509643555"
21434,FYAK,small_airport,Aussenkehr Airport,970.0,AF,NaN,NA-KA,Aussenkehr,FYAK,NaN,NaN,"-28.4587, 17.4645"
21435,FYAM,small_airport,Aminuis Airstrip,4012.0,AF,NaN,NA-OH,Aminuis,FYAM,NaN,NaN,"-23.655799865722656, 19.351699829101562"
21436,FYAR,medium_airport,Arandis Airport,1905.0,AF,NaN,NA-ER,Arandis,FYAR,ADI,NaN,"-22.462200164794922, 14.979999542236328"
...,...,...,...,...,...,...,...,...,...,...,...,...
35695,NA-0172,small_airport,Haklesdoorn Airport,NaN,AF,NaN,NA-KA,Haklesdoorn,NaN,NaN,NaN,"-28.886092, 18.239166"
35696,NA-0173,small_airport,Ndonga Linena Airstrip,NaN,AF,NaN,NA-KW,Shitemo,NaN,NaN,NaN,"-17.98345, 20.534708"
35697,NA-0174,small_airport,Farm Etemba Landing Strip,3503.0,AF,NaN,NA-ER,Etemba Wilderness Camp,NaN,NaN,NaN,"-21.405111, 15.606778"
35698,NA-0175,small_airport,Farm Dawib Ost Landing Strip,3503.0,AF,NaN,NA-ER,Hohenstein Lodge,NaN,NaN,NaN,"-21.774528, 15.515361"


In [50]:
first_flag = True
for filename in glob.glob(r'GeospacialDBs_Data/flightlist_*.csv'):
    df_flight = pd.read_csv(filename, header=0, sep=",",  dtype={"number": str})
    country_map = {ident:iso_country for ident, iso_country in zip(df_airport_codes["ident"],df_airport_codes["iso_country"])}
    daily_grouped = df_flight.groupby(['origin', "day"])["callsign"].agg(['count'])
    daily_index = daily_grouped.index.get_level_values(0)
    uniques = df_airport_codes["ident"].unique()
    daily_grouped['iso_country'] = daily_grouped.index.get_level_values(0).map(country_map)
    country_daily_origin = daily_grouped.groupby(['iso_country', "day"])['count'].agg(['count'])
    daily_grouped2 = df_flight.groupby(['destination', "day"])["callsign"].agg(['count'])
    daily_index2 = daily_grouped2.index.get_level_values(0)
    uniques = df_airport_codes["ident"].unique()
    daily_grouped2['iso_country'] = daily_grouped2.index.get_level_values(0).map(country_map)
    country_daily_destination = daily_grouped2.groupby(['iso_country', "day"])["count"].agg(['count'])
    country_daily = pd.concat([country_daily_origin, country_daily_destination], axis=1)
    country_daily.columns = ['origin_count', 'destination_count']
    if first_flag:
        country_flights_io = country_daily
        first_flag = False
    else:
        country_flights_io = country_flights_io.append([country_daily])
country_flights_io.head(10)

origin_count  destination_count
iso_country day                                                       
AE          2020-01-01 00:00:00+00:00           6.0                9.0
            2020-01-02 00:00:00+00:00           8.0                9.0
            2020-01-03 00:00:00+00:00           9.0               11.0
            2020-01-04 00:00:00+00:00           7.0               15.0
            2020-01-05 00:00:00+00:00           9.0               14.0
            2020-01-06 00:00:00+00:00           7.0               11.0
            2020-01-07 00:00:00+00:00           7.0               12.0
            2020-01-08 00:00:00+00:00           8.0               12.0
            2020-01-09 00:00:00+00:00           8.0               12.0
            2020-01-10 00:00:00+00:00           7.0               12.0

In [51]:
country_flights_io.loc["AE"]

,origin_count,destination_count
day,,
2020-01-01 00:00:00+00:00,6.0,9.0
2020-01-02 00:00:00+00:00,8.0,9.0
2020-01-03 00:00:00+00:00,9.0,11.0
2020-01-04 00:00:00+00:00,7.0,15.0
2020-01-05 00:00:00+00:00,9.0,14.0
...,...,...
2020-12-27 00:00:00+00:00,8.0,14.0
2020-12-28 00:00:00+00:00,8.0,12.0
2020-12-29 00:00:00+00:00,11.0,10.0


# Iso Codes for translating to country names

In [52]:
df_iso_codes = pd.read_csv(r'GeospacialDBs_Data/wikipedia-iso-country-codes.csv', header=0, sep=",")
df_iso_codes.head(5)
    


,English short name lower case,Alpha-2 code,Alpha-3 code,Numeric code,ISO 3166-2
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX
2,Albania,AL,ALB,8,ISO 3166-2:AL
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ
4,American Samoa,AS,ASM,16,ISO 3166-2:AS


In [53]:
country_flights_io[country_flights_io.isnull().any(axis=1)]

origin_count  destination_count
iso_country day                                                       
BA          2020-01-02 00:00:00+00:00           NaN                2.0
            2020-01-04 00:00:00+00:00           NaN                2.0
            2020-01-05 00:00:00+00:00           NaN                1.0
            2020-01-06 00:00:00+00:00           NaN                2.0
            2020-01-07 00:00:00+00:00           NaN                1.0
...                                             ...                ...
VE          2020-12-14 00:00:00+00:00           NaN                1.0
            2020-12-16 00:00:00+00:00           NaN                1.0
            2020-12-25 00:00:00+00:00           NaN                1.0
            2020-12-27 00:00:00+00:00           NaN                1.0
            2020-12-29 00:00:00+00:00           NaN                1.0

[2112 rows x 2 columns]

In [54]:
country_flights_io=country_flights_io.reset_index()
country_flights_io

,iso_country,day,origin_count,destination_count
0,AE,2020-01-01 00:00:00+00:00,6.0,9.0
1,AE,2020-01-02 00:00:00+00:00,8.0,9.0
2,AE,2020-01-03 00:00:00+00:00,9.0,11.0
3,AE,2020-01-04 00:00:00+00:00,7.0,15.0
4,AE,2020-01-05 00:00:00+00:00,9.0,14.0
...,...,...,...,...
33128,ZA,2020-12-27 00:00:00+00:00,10.0,10.0
33129,ZA,2020-12-28 00:00:00+00:00,10.0,13.0
33130,ZA,2020-12-29 00:00:00+00:00,8.0,13.0
33131,ZA,2020-12-30 00:00:00+00:00,10.0,10.0


In [55]:
country_flights_io.isnull().sum()

iso_country             0
day                     0
origin_count         1676
destination_count     436
dtype: int64

In [56]:
country_flights_io.rename(columns={'iso_country':'country_region_code','day':'date'},inplace=True)
country_flights_io

,country_region_code,date,origin_count,destination_count
0,AE,2020-01-01 00:00:00+00:00,6.0,9.0
1,AE,2020-01-02 00:00:00+00:00,8.0,9.0
2,AE,2020-01-03 00:00:00+00:00,9.0,11.0
3,AE,2020-01-04 00:00:00+00:00,7.0,15.0
4,AE,2020-01-05 00:00:00+00:00,9.0,14.0
...,...,...,...,...
33128,ZA,2020-12-27 00:00:00+00:00,10.0,10.0
33129,ZA,2020-12-28 00:00:00+00:00,10.0,13.0
33130,ZA,2020-12-29 00:00:00+00:00,8.0,13.0
33131,ZA,2020-12-30 00:00:00+00:00,10.0,10.0


In [57]:
country_flights_io.set_index(['country_region_code','date'],inplace=True)

In [58]:
country_flights_io

origin_count  destination_count
country_region_code date                                                      
AE                  2020-01-01 00:00:00+00:00           6.0                9.0
                    2020-01-02 00:00:00+00:00           8.0                9.0
                    2020-01-03 00:00:00+00:00           9.0               11.0
                    2020-01-04 00:00:00+00:00           7.0               15.0
                    2020-01-05 00:00:00+00:00           9.0               14.0
...                                                     ...                ...
ZA                  2020-12-27 00:00:00+00:00          10.0               10.0
                    2020-12-28 00:00:00+00:00          10.0               13.0
                    2020-12-29 00:00:00+00:00           8.0               13.0
                    2020-12-30 00:00:00+00:00          10.0               10.0
                    2020-12-31 00:00:00+00:00           7.0               12.0

[33133 rows x 2 columns]

In [59]:
#replace NaNs with Zeros
country_flights_io.fillna(0,inplace=True)

In [60]:
country_flights_io.to_csv(r'GeospacialDBs_Data/Flights_io_per_day_and_country.csv', index = True)